# 5. Optimization methods
Optimization methods attempt to find the minimum or maximum of functions. Here the steepest descent, conjugate gradient, Newton-Raphson, and Broyden-Fletcher-Goldfarb-Shanno methods are presented.

Functions with two variables can be visualized as a surface (i.e., a contour map) where the lowest and highest point on the map corresponds to the function’s max and min.  Although higher dimensional functions cannot be readily visualized, the analogy of traversing a surface is still convenient. Using the map concept, one can appreciate the general features of these algorithms. 

1.	Select an initial guess, which is a point on the surface
2.	Determine a direction to move
3.	Take a step by moving along the determined direction
4.	Test for convergence (Is the difference between the function’s values for the last two points less than a threshold value?)
5.	Repeat steps 2 to 4 to continue searching surface


The most significant difference in the algorithms presented in this notebook is how each defines the step direction. The various approaches are described in the following sections.  It is important to emphasize that numerical optimizations can be extremely time consuming.  For the optimization to complete in a reasonable amount of time, the threshold for the convergence criteria must be selected with care.  In addition, an optimization algorithm should always limit the number of iterations so as not to get stuck in an endless search.  Lastly, the method should keep track of the step size because an indication that the search is proceeding successfully is a decrease in successive step sizes.

# 5.1 Steepest Descent

The steepest descent method is the simplest iterative method of optimization method. Steepest descent makes iterative steps in the direction of greatest decrease. A function’s gradient at a point determines the direction where the rate of changes is the highest.  Thus the opposite direction will be the steepest decent.

For an initial guess, $x_0$, the search direction, $sd_0$, that is in the direction of steepest descent becomes

 $$ sd_0 = -\nabla f(x_0)$$

A step is taken by using a line search minimization along this search direction, 

$$ x_i = x_0 + \lambda sd_0 $$

where $\lambda$ minimizes $f(x)$ in the search direction. For simplicity, the details of the of the minimization are ignored.

A new search direction is constructed as the direction of steepest descent at the new point.

 $$ sd_i = -\nabla f(x_i)$$
 
 
As an example, the cell directly below implements steepest descent on the function,
 $$ f(x,y) = sin(0.5x^2-0.25y^2+3)*cos(2x+1-e^y)$$
Following the code is an image depicting steepest descent implemented on the surface of this function.


In [1]:
###########
# IMPORTS #
###########
import numpy as np
from scipy.optimize import line_search as linesearch
import scipy.linalg as spla
import scipy as sp
import plotting_functions as pf
import matplotlib.pyplot as plt
import sympy as syp
%matplotlib notebook
#############
# FUNCTIONS #
#############


def f2(x, y):
    return syp.sin(0.5 * x**2 - 0.25 * y**2 + 3) * syp.cos(2 * x + 1 - syp.exp(y))


def evaluate_hessian(f):
    x,y= syp.symbols('x y')
    fhess=np.array([[None for i in range(2)],[None for i in range(2)]])
    for idx1,v1 in enumerate((x,y)):
        for idx2,v2 in enumerate((x,y)):
                fhess[idx1][idx2]=syp.diff(f(x,y), v1, v2)
    fhess_lambda = syp.lambdify((x,y),fhess,'numpy')
    return(fhess_lambda)


def evaluate_gradient(f):
    x,y= syp.symbols('x y')
    fgrad=np.array([None for i in range(2)])
    for idx1,v1 in enumerate((x,y)):
                fgrad[idx1]=syp.diff(f(x,y), v1)
    fgrad_lambda = syp.lambdify((x,y),fgrad,'numpy')
    return fgrad_lambda

def compact_input(f):
    return lambda X: np.array(f(X[0],X[1]),dtype=float)
gradient = evaluate_gradient(f2)
Gradient = compact_input(gradient)
hessian = evaluate_hessian(f2)
Hessian = compact_input(hessian)
f = compact_input(f2)

##########
# GLOBAL VARS
##########
# initial guess
initial_guess = np.array((1.6, -1.5))  
print(initial_guess)
f(initial_guess)
# convergence criteria
residual_tolerance = 1e-6  
update_tolerance = 1e-5
IT_MAX = 1000
# domain for plotting
xminimum = -1
xmaximum = 2
yminimum = -4
ymaximum = 1
#data structures to summarize resules
steps_per_method = dict()
difference_per_method = dict()


[ 1.6 -1.5]


In [2]:
##########
#MAIN CODE
##########
old_guess = initial_guess
search_direction = -Gradient(old_guess) # set the search direction to the negative gradient at the initial guess
converged = False 
num_steps = 0
steps_x =[]
steps_y =[]
steps_x.append(old_guess[0])
steps_y.append(old_guess[1])
#repeat until convergenced or exceed IT_MAX
while not converged and num_steps <= IT_MAX: 
    print("STEP {:d}".format(num_steps))
    num_steps+=1
    gamma = linesearch(f,Gradient,old_guess,search_direction)[0] # find the step size that minimizes f along the search direction
    if gamma == None: 
        print("ERROR: No appropriate step size found. Exiting minimization")
        exit()
    else:
        new_guess = old_guess + gamma*search_direction # update the current guess
        search_direction = -Gradient(new_guess) # set the new search direction to the negative gradient at the new guess
        diff = old_guess-new_guess # calculate the vector difference of the old guess and new guess
        diff = np.sqrt(np.dot(diff,diff)) # calculate the magnitude of the separation between the old guess and the new guess
        print("OLD GUESS: {}".format(old_guess))
        print("NEW GUESS: {}".format(new_guess))
        print("DIFFERENCE: {}".format(diff))
        old_guess = new_guess # update the old guess so we can take another step
        steps_x.append(old_guess[0])
        steps_y.append(old_guess[1])
        if(diff<residual_tolerance): # if the step was small enough consider this the minimum
            converged=True
steps_x = np.array(steps_x)
steps_y = np.array(steps_y)
steps_per_method["Steepest Descent"] = num_steps
difference_per_method["Steepest Descent"] = diff


STEP 0
OLD GUESS: [ 1.6 -1.5]
NEW GUESS: [ 1.32197638 -3.03485489]
DIFFERENCE: 1.5598322571776573
STEP 1
OLD GUESS: [ 1.32197638 -3.03485489]
NEW GUESS: [ 1.09237061 -3.02949091]
DIFFERENCE: 0.2296684169793337
STEP 2
OLD GUESS: [ 1.09237061 -3.02949091]
NEW GUESS: [ 1.16003733 -2.93884683]
DIFFERENCE: 0.11311557988600016
STEP 3
OLD GUESS: [ 1.16003733 -2.93884683]
NEW GUESS: [ 1.10514384 -2.91206942]
DIFFERENCE: 0.061076388592093266
STEP 4
OLD GUESS: [ 1.10514384 -2.91206942]
NEW GUESS: [ 1.12040319 -2.88218061]
DIFFERENCE: 0.033558741890529604
STEP 5
OLD GUESS: [ 1.12040319 -2.88218061]
NEW GUESS: [ 1.10175656 -2.87314903]
DIFFERENCE: 0.020718741336484377
STEP 6
OLD GUESS: [ 1.10175656 -2.87314903]
NEW GUESS: [ 1.10688367 -2.86270543]
DIFFERENCE: 0.011634255693789298
STEP 7
OLD GUESS: [ 1.10688367 -2.86270543]
NEW GUESS: [ 1.10040725 -2.8595727 ]
DIFFERENCE: 0.007194303591980734
STEP 8
OLD GUESS: [ 1.10040725 -2.8595727 ]
NEW GUESS: [ 1.10218593 -2.85591147]
DIFFERENCE: 0.004070418445

In [3]:
pf.plot_func(xminimum, xmaximum, yminimum, ymaximum, steps_x, steps_y, f)

AttributeError: 'ImmutableDenseNDimArray' object has no attribute 'could_extract_minus_sign'

<Figure size 1008x475.2 with 0 Axes>

# 5.2 Conjugate Gradient #

Conjugate gradient is very similar to the  steepest descent method, however the method attempts to ensure previous progress is not undone with future search steps. The added restriction is called conjugacy.Again, for an initial guess, $x_0$ the starting search direction, $sd_0$ is taken as the negative gradient.

 $$ sd_0 = -\nabla f(x_0)$$

The line search minimization along this search direction provides the step size. All new search directions are constructed by including a term to prevent successive steps from diminishing progress made from the previous search direction.   This second term is determined through a process similar to orthogonalization.

$$ sd_{i+1} = -\nabla f(x_{i+1}) + \frac{(-\nabla f(x_{i+1})) \bullet (-\nabla f(x_{i+1})) }{(-\nabla f(x_{i})) \bullet (-\nabla f(x_{i}))} sd_{i} $$

In practice, the line search minimization may fail because the search direction may point uphill.
This can be resolved by resetting the search direction to point in the direction of steepest descent.

The code below applies the conjugate gradient method to the example function and plots the search path.

In [ ]:
#############
# MAIN CODE #
#############
converged = False
num_steps = 0
old_guess = initial_guess
old_grad = -Gradient(old_guess)
# set first search direction in the direction of steepest descent
old_search_direction = old_grad
steps_x = []
steps_y = []
steps_x.append(old_guess[0])
steps_y.append(old_guess[1])
while not converged and num_steps <= IT_MAX:
    print("STEP {:d}".format(num_steps))
    alpha = linesearch(f, Gradient, old_guess, old_search_direction)[0]
    if alpha == None:
        print("WARNING: LINE SEARCH DID NOT CONVERGE... RESETTING SEARCH DIRECTION TO STEEPEST DESCENT!")
        old_search_direction = -Gradient(old_guess)
        alpha = linesearch(f, Gradient, old_guess, old_search_direction)[0]  # step size
    new_guess = old_guess + alpha * old_search_direction
    new_grad = -Gradient(new_guess)
    # gamma encompasses the fraction in the above equation
    gamma = np.dot(new_grad - old_grad, new_grad) / np.dot(old_grad, old_grad)
    # update the search direction
    new_search_direction = new_grad + np.dot(gamma, old_search_direction)
    diff = old_guess - new_guess
    diff = np.sqrt(np.dot(diff, diff))
    print("OLD GUESS: {}".format(old_guess))
    print("NEW GUESS: {}".format(new_guess))
    print("DIFFERENCE: {}".format(diff))
    num_steps += 1
    old_guess = new_guess
    old_grad = new_grad
    old_search_direction = new_search_direction
    steps_x.append(old_guess[0])
    steps_y.append(old_guess[1])
    if(diff < residual_tolerance):
        converged = True
steps_x = np.array(steps_x)
steps_y = np.array(steps_y)
steps_per_method["Conjugate Gradient"] = num_steps
difference_per_method["Conjugate Gradient"] = diff


In [ ]:
pf.plot_func(xminimum, xmaximum, yminimum, ymaximum, steps_x, steps_y, f)

# 5.3 Newton-Raphson

In the root-finding section of these lessons, the Newton-Raphson root finding method was described for a one-dimensional system. This method can be extended to higher dimensions. As a reminder, the Newton-Raphson method takes iterative steps:

$$x_{i+1}\ =\ x_i - \frac{f(x_i)}{f'(x_i)}$$

To minimize the first derivative to find an extremum, the following iterative equation could be used:
$$x_{i+1}\ =\ x_i - \frac{f'(x_i)}{f''(x_i)}$$

To optimize a higher order function, the Hessian matrix is introduced.
The Hessian is a square matrix composed of second derivatives that describe the local curvature of a function. The iterative steps are given by:
$$ p_i = (x_i,y_i) $$
and
$$p_{i+1} = p_i - \gamma[H(p_i)]^{-1}]\nabla f(p_i)$$

where the step size, $\gamma$, is found using a line search and $H(p_i)$ is the Hessian:
$$H(p_i)= \ \begin{bmatrix}
{\frac{\partial^2 f(x_i,y_i)}{\partial x^2}} 
& {\frac{\partial^2 f(x_i,y_i)}{\partial x \partial y}} \\
{\frac{\partial^2 f(x_i,y_i)}{\partial y \partial x}} 
& {\frac{\partial^2 f(x_i,y_i)}{\partial y^2}} 
\end{bmatrix} $$


It is important to note that the Newton-Raphson method requires the Hessian to be positive definite, which ensures a local minimum near $p_i$ is isolated and not a saddle point or local maximum. We will see below that the Newton-Raphson method finds the minimum of a well-behaved, convex function, such as $x^2\ +\ y^2$. If we solve for the Hessian of this function:
$$Hf(x,y)= \ \begin{bmatrix}
{\frac{\partial^2 f(x,y)}{\partial x^2}} 
& {\frac{\partial^2 f(x,y)}{\partial x \partial y}} \\
{\frac{\partial^2 f(x,y)}{\partial y \partial x}} 
& {\frac{\partial^2 f(x,y)}{\partial y^2}} 
\end{bmatrix} $$
$$Hf(x,y)= \ \begin{bmatrix}
{\frac{\partial^2 (x^2+y^2)}{\partial x^2}} 
& {\frac{\partial^2 (x^2+y^2)}{\partial x \partial y}} \\
{\frac{\partial^2 (x^2+y^2)}{\partial y \partial x}} 
& {\frac{\partial^2 (x^2+y^2)}{\partial y^2}} 
\end{bmatrix} $$

$$Hf(x,y)= \ \begin{bmatrix}
2
& 0 \\
0 
& 2 
\end{bmatrix} $$
These positive values of the Hessian indicate that our function has a local minimum, thus Newton's method of optimization will perform well.
Despite the requirement of a positive definite Hessian, the Newton-Raphson's method has excellent convergence since most functions near a minimum resemble $x^2\ +\ y^2$.

The code below applies the Newton-Raphson optimization method to the example function and plots the optimization path.

In [ ]:
##########
# MAIN CODE
##########
old_guess = initial_guess
old_search_direction = -Gradient(old_guess)
num_steps = 0
converged = False
steps_x = []
steps_y = []
steps_x.append(old_guess[0])
steps_y.append(old_guess[1])
while not converged and num_steps <= IT_MAX:
    print("STEP {:d}".format(num_steps))
    num_steps += 1
    A = Hessian(old_guess)
    try:
        if num_steps == 1:
            new_search_direction = -Gradient(old_guess)
            gamma = linesearch(
                f, Gradient, old_guess, new_search_direction)[0]  # step size
            new_guess = old_guess + gamma * new_search_direction
        else:
            new_search_direction = spla.solve(A, -Gradient(old_guess))
            new_guess = old_guess + new_search_direction
    except:
        print("WARNING: HESSIAN IS NOT POSITIVE DEFINITE... RESETTING SEARCH DIRECTION TO STEEPEST DESCENT!")
        new_search_direction = -Gradient(old_guess)
        gamma = linesearch(f, Gradient, old_guess, new_search_direction)[
            0]  # step size
        new_guess = old_guess + gamma * new_search_direction
    diff = old_guess - new_guess
    diff = np.sqrt(np.dot(diff, diff))
    print("OLD GUESS: {}".format(old_guess))
    print("NEW GUESS: {}".format(new_guess))
    print("DIFFERENCE: {}".format(diff))
    old_guess = new_guess
    steps_x.append(old_guess[0])
    steps_y.append(old_guess[1])
    old_search_direction = new_search_direction
    if(diff < residual_tolerance):
        converged = True
steps_x = np.array(steps_x)
steps_y = np.array(steps_y)
steps_per_method["Newton-Raphson"] = num_steps
difference_per_method["Newton-Raphson"] = diff


In [ ]:
pf.plot_func(xminimum, xmaximum, yminimum, ymaximum, steps_x, steps_y, f)

# 5.4 Broyden-Fletcher-Goldfarb-Shanno (BFGS)

The BFGS method is one of the quasi-Newton methods and, as the name suggests, approximates the Newton-Raphson method discussed above. The major difference of a quasi-Newton method is that the Hessian is not calculated directly, but instead approximated. This approximation results in the BFGS method being more robust and often times converging more quickly than the Newton-Raphson method.
   
 The only difference to the Newton method is the update of the approximate Hessian, $H_{approx}$ and the update of the search direction. The mathematics of these updates are out of the scope of these notebooks; however, the interested reader is directed towards *Polak, E. Computational Methods in Optimization; a Unified Approach. New York: Academic, 1971. Print.* for a more detailed explanation.
 
 The code below applies the BFGS method to the example function and plots the optimization path.

In [ ]:
##########
# MAIN CODE
##########
old_guess = initial_guess

approx_hessian = np.identity(2)
inverse_approx_hessian = np.identity(2)

num_steps = 0

s_i = np.array([1.0, 0.0])
y_i = np.array([0.0, 1.0])

converged = False
steps_x = []
steps_y = []
steps_x.append(old_guess[0])
steps_y.append(old_guess[1])
while not converged and num_steps <= IT_MAX:
    print("STEP {:d}".format(num_steps))
    num_steps += 1
    new_search_direction = np.dot(inverse_approx_hessian, -Gradient(old_guess))
    try:
        s_i = linesearch(f, Gradient, old_guess, new_search_direction)[
            0] * new_search_direction
    except:
        print("WARNING: HESSIAN IS NOT POSITIVE DEFINITE... RESETTING SEARCH DIRECTION TO STEEPEST DESCENT!")
        new_search_direction = -Gradient(old_guess)
        s_i = linesearch(f, Gradient, old_guess, new_search_direction)[
            0] * new_search_direction

    new_guess = old_guess + s_i
    y_i = Gradient(new_guess) - Gradient(old_guess)
    approx_hessian += ((np.outer(y_i, y_i)) / (np.dot(y_i, s_i))) - ((np.dot(approx_hessian,
                                                                             np.dot(np.outer(s_i, s_i), approx_hessian))) / (np.dot(s_i, np.dot(approx_hessian, s_i))))
    # update using Sherman-Morrison formula
    part_A = np.dot(s_i, y_i)
    part_A += np.dot(y_i, np.dot(inverse_approx_hessian, y_i))
    part_A /= np.dot(s_i, y_i)**2
    part_A *= np.outer(s_i, s_i)
    part_B = np.dot(inverse_approx_hessian, np.outer(y_i, s_i))
    part_B += np.dot(np.outer(s_i, y_i), inverse_approx_hessian)
    part_B /= np.dot(s_i, y_i)
    inverse_approx_hessian += part_A - part_B

    diff = old_guess - new_guess
    diff = np.sqrt(np.dot(diff, diff))
    print ("OLD GUESS: {}".format(old_guess))
    print ("NEW GUESS: {}".format(new_guess))
    print ("DIFFERENCE: {}".format(diff))
    old_guess = new_guess
    old_search_direction = new_search_direction
    steps_x.append(old_guess[0])
    steps_y.append(old_guess[1])
    if(diff < residual_tolerance):
        converged = True
steps_x = np.array(steps_x)
steps_y = np.array(steps_y)
steps_per_method["BFGS"] = num_steps
difference_per_method["BFGS"] = diff


In [ ]:
pf.plot_func(xminimum, xmaximum, yminimum, ymaximum, steps_x, steps_y, f)

# 5.5 Summary of Results

In [ ]:
print("{:->39}".format(""))
print("{:>20} : {:>5} {:>10}".format("Method", "Steps", "Difference"))
print("{:->20}   {:->5} {:->10}".format("", "", ""))
for i in sorted(difference_per_method, key=difference_per_method.get, reverse=True):
    print("{:>20} : {:>5} {:>6.4e}".format(i, steps_per_method[i], difference_per_method[i]))
print("{:->39}".format(""))


When deciding on an optimization method to use, we consider (above) the number of steps required to converge. The Newton-Raphson method converges quickly, and with great accuracy. However the Newton-Raphson method requires the calculation of a Hessian, which is expensive. The BFGS method avoids the Hessian calculation and instead iteratively builds an approximate Hessian. The BFGS method also converges quickly and is widely used in real world applications. The conjugate gradient method requires more steps to converge, but is accurate. Finally, the steepest descent method, while slow, guarantees convergence.

# 5.6 Application

## Your Task

Optimization methods are commonly used in chemistry to find the lowest energy structure of a molecule, which corresponds to the equilibrium gas phase geometry. The basic approach is to use a computational method to evaluate the energy and gradients with respect to moves of the nuclei. This energy surface can be explored using optimization methods to locate the ideal geometry. 

The code box below uses the BFGS method to optimize the Hartree-Fock energy of a ring of 6 hydrogen atoms. The two parameters that will be optimized are the radius of the ring and the ratio between pairs of hydrogen atoms. Remember the BFGS method requires first derivatives, and in this case the first derivate is calculated numerically using the central difference method.

There are two stable geometries; one with equally spaced hydrogens and another with "dimers" of hydrogens.

Run the first cell to perform the optimization, and then run the second cell to visualize the optimization.

In [ ]:
# DIMERS   
# radius = 4.5 # atomic units
# deformation = 0.65 # ratio (must be less than or equal to 1)
# RING
radius = 2.0 # atomic units
deformation = 1.0 # ratio (must be less than or equal to 1)

if deformation > 1.0 or deformation < 0.0:
        deformation = np.abs(deformation) % 1.0

steps_radius, steps_deformation, steps_energy = pf.optimize_H6_ring(radius, deformation)
        

In [ ]:
import ipywidgets as ipw
from ipywidgets import *
step_set = ipw.IntSlider(min=0,max =len(steps_energy)-1,value=0)
interactive_plot = ipw.interactive(pf.plotting, step=step_set,steps_radius=fixed(steps_radius), steps_energy=fixed(steps_energy),steps_deformation=fixed(steps_deformation))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

### Question:
Please insert the number of the correct answer into the function below.
Based on the energies of the two structures calculated above the ring with equally paired hydrogens is  
    1.) a global minimum
    2.) a local minimum
    3.) an equivilant structure to the paired hydrogens
    4.) a maximum
    5.) None of the above

In [ ]:
pf.question_one_check()